In [ ]:
import os
import pandas as pd
import numpy as np
import argparse
from tqdm import tqdm


# TODO are we interested in:
# 'non_coding_transcript_variant', 'frameshift_variant', or 'PolyPhenCat' 'Unknown' category?
cats = {
    "LoF": np.array(["LoF_HC", "LoF_LC"]),
    "Consequence": np.array(['3_prime_UTR_variant',
                     '5_prime_UTR_variant',
                     'TF_binding_site_variant',
                     'downstream_gene_variant',
                     'intergenic_variant',
                     'intron_variant',
                     'missense_variant',
                     'regulatory_region_variant',
                     'splice_acceptor_variant',
                     'splice_donor_variant',
                     'splice_region_variant',
                     'stop_gained',
                     'synonymous_variant',
                     'upstream_gene_variant',
                     'frameshift_variant',
                    ]),
}


def to_cat(var, cat):
    if len(var) and cat == 'LoF': var = f'LoF_{var}'
    return pd.DataFrame(np.isin(cats[cat],var).astype(int).reshape(1,-1), columns=cats[cat])

def get_header():
    header = ["SubjectID", "GeneName", "Chromosome", "Position", "Ref", "Alt", "AF"]
    header.extend(cats["Consequence"])
    header.extend(cats["LoF"])
    return "\t".join(header) + "\n"

def filelen(path):
    fname = path.split("/")[-1]
    os.system(f"wc -l {path} > /tmp/wc_{fname}")
    n = int(open(f"/tmp/wc_{fname}", "r").readlines()[0].split()[0])
    return n    

def get_vep(l):
    if 'CSQ=' not in l[7]: return None

    # TODO read this from the file, right now I am stripping that part of the header.
    vep_fields = 'Allele|Consequence|IMPACT|SYMBOL|Gene|Feature_type|Feature|BIOTYPE|EXON|INTRON|HGVSc|HGVSp|cDNA_position|CDS_position|Protein_position|Amino_acids|Codons|Existing_variation|DISTANCE|STRAND|FLAGS|SYMBOL_SOURCE|HGNC_ID|LoF|LoF_filter|LoF_flags|LoF_info'.split("|")
    lidx = np.where(np.array(vep_fields) == "LoF")[0][0]
    vidx = np.where(np.array(vep_fields) == "Consequence")[0][0]
    gidx = np.where(np.array(vep_fields) == "Gene")[0][0]
    
    vep_infos = l[7].split('CSQ=')[1].split(',')
    vep_infos = [v.split('|') for v in vep_infos]

    vep_df = pd.DataFrame(vep_infos, columns=vep_fields)

    # convert to categorical
    vep_cat_df = pd.DataFrame()
    for v in vep_df.index:
        var = to_cat(vep_df.loc[v]["Consequence"], "Consequence")
        lof = to_cat(vep_df.loc[v]["LoF"], "LoF")
        cats = pd.concat([var, lof], axis=1)
        vep_cat_df = pd.concat([vep_cat_df, cats], axis=0)
    vep_cat_df.index = vep_df.loc[:,"Gene"]

    # aggregate multiple annotations over same gene using max
    for g in vep_cat_df.index.unique():
        if len(vep_cat_df.loc[[g]]) > 1:
            df2 = pd.DataFrame(vep_cat_df.loc[[g]].max(0)).T
            df2.index = [g]
            vep_cat_df = pd.concat([vep_cat_df.drop(g), df2])

    return vep_cat_df


def process_line(l, cols):
    
    l = l.strip("\n").split()
    rare_idx = np.where(np.isin(l, ["0/1", "1/0", "1/1", "0|1", "1|0", "1|1"]))[0]
    rare_ids = [cols[i] for i in rare_idx]
    af = l[7].split("AFR_AF=")[1].split(";")[0]
    chidx = np.where(np.array(cols) == "CHROM")[0][0]
    pidx = np.where(np.array(cols) == "POS")[0][0]
    refidx = np.where(np.array(cols) == "REF")[0][0]
    altidx = np.where(np.array(cols) == "ALT")[0][0]
    ch,pos,ref,alt = l[chidx], l[pidx], l[refidx], l[altidx]

    vep_cat_df = get_vep(l)
    if vep_cat_df is None: return None

    # Create annotation line for each id, gene pair
    out_lines = []
    for rid in set(rare_ids):
        for gene in vep_cat_df.index:
            out_line = [rid, gene, ch, pos, ref, alt, af]
            out_line.extend(vep_cat_df.loc[gene].array.astype(str).tolist())
            out_lines.append("\t".join(out_line) + "\n")

    genes = vep_cat_df.index.array
    return "".join(out_lines), genes, rare_ids

    
def vcf_to_tsv(in_file, out_dset, out_gene_outliers):
    
    vcf = open(in_file, 'r')
    cols = vcf.readline().strip("#").split()
    print(cols)
    n = filelen(in_file)    
  
    out = open(out_dset, 'w')
    gout = open(out_gene_outliers, 'w')
               
    header = get_header()
    out.write(header)

    for l in tqdm(vcf, total=n):
        out_line, genes, rid = process_line(l, cols)
        if out_line is not None and out_line != "":
            out.write(out_line)
            for gene in genes:
                gout.write("\t".join([gene] + rid) + "\n")

        
if __name__ == "__main__":

    
#     argParser = argparse.ArgumentParser()
#     argParser.add_argument("--pop", default="AFR", type=str)
#     argParser.add_argument("--vep_dir", default='/oak/stanford/groups/smontgom/erobb/data/vep', type=str)
#     argParser.add_argument("--out_dir", default='/oak/stanford/groups/smontgom/erobb/data/watershed', type=str)
#     args = argParser.parse_args()

#     tsv_out = f'{args.out_dir}/AF.all.{args.pop}.hg38a.ID.ba.VEP.rare.ws.tsv'
#     gene_outliers = f'f"{args.out_dir}/gene_outliers_{args.pop}.tsv'
#     vcf_in = f"{args.vep_dir}/AF.all.{args.pop}.hg38a.ID.ba.VEP.rare.vcf"
    
    pop = 'ESN'
    vep_dir = '/oak/stanford/groups/smontgom/erobb/data/vep'
    out_dir = "/tmp" 
    tsv_out = f'{out_dir}/AF.all.{pop}.hg38a.ID.ba.VEP.rare.ws.tsv'
    gene_outliers = f'{out_dir}/gene_outliers_{pop}.tsv'
    vcf_in = f"{vep_dir}/AF.all.{pop}.hg38a.ID.ba.VEP.rare.vcf"
    
    vcf_to_tsv(vcf_in, tsv_out, gene_outliers)

        

['CHROM', 'POS', 'ID', 'REF', 'ALT', 'QUAL', 'FILTER', 'INFO', 'FORMAT', 'HG02941', 'HG02976', 'HG03114', 'HG03135', 'HG03189', 'HG03271', 'HG03313', 'HG03367', 'HG02943', 'HG02977', 'HG02979', 'HG03115', 'HG03136', 'HG03190', 'HG03279', 'HG03280', 'HG03342', 'HG03369', 'HG02981', 'HG03117', 'HG03139', 'HG03291', 'HG03343', 'HG03370', 'HG02946', 'HG03118', 'HG03157', 'HG03195', 'HG03294', 'HG03351', 'HG03372', 'HG02947', 'HG03099', 'HG03120', 'HG03159', 'HG03196', 'HG03295', 'HG03352', 'HG03499', 'HG02952', 'HG03100', 'HG03121', 'HG03124', 'HG03160', 'HG03198', 'HG03297', 'HG03354', 'HG03511', 'HG02953', 'HG03103', 'HG03123', 'HG03126', 'HG03162', 'HG03199', 'HG03514', 'HG02968', 'HG03105', 'HG03127', 'HG03163', 'HG03202', 'HG03300', 'HG03515', 'HG02970', 'HG03108', 'HG03129', 'HG03166', 'HG03265', 'HG03301', 'HG03517', 'HG02922', 'HG02971', 'HG03109', 'HG03130', 'HG03168', 'HG03169', 'HG03267', 'HG03303', 'HG03518', 'HG02923', 'HG02973', 'HG03111', 'HG03172', 'HG03268', 'HG03304', 'HG

 62%|██████████████████████████████████████▍                       | 1048598/1690514 [2:40:19<1:24:14, 127.00it/s]

### filter out IDs with too much variance

### cadd

In [ ]:

cats = {
    # TODO these are strange, why does PolyPhen include 'benign' and 'unknown' as categories but SIFT doesn't
    "SIFTcat": np.array(["deleterious", "tolerated"]),
    "PolyPhenCat": np.array(["benign", "possibly_damaging", "unknown"]),
    "LoF": np.array(["LoF_HC", "LoF_LC"]),
    "Consequence": np.array(['3_prime_UTR_variant',
                     '5_prime_UTR_variant',
                     'TF_binding_site_variant',
                     'downstream_gene_variant',
                     'intergenic_variant',
                     'intron_variant',
                     'missense_variant',
#                      'non_coding_transcript_variant',
#                      'non_coding_transcript_exon_variant',
                     'regulatory_region_variant',
                     'splice_acceptor_variant',
                     'splice_donor_variant',
                     'splice_region_variant',
                     'stop_gained',
                     'synonymous_variant',
                     'upstream_gene_variant',
                     'frameshift_variant',
                    ]),
}


def query_tabix(ch, s, e, opt=""):
    os.system(f"tabix -h {CADD_FILE} {ch}:{s}-{e} > /tmp/{ch}:{s}-{e}.tsv")
    return pd.read_table(f"/tmp/{ch}:{s}-{e}.tsv", header=1)


def get_cadd(ch,pos,ref,alt):
    # Only supports SNVs at this time
    if not (len(ref) == 1 and len(alt)== 1):
        return [""] * len(cadd_anno)

    # Query tabix
    cadd_table = query_tabix(ch, pos, int(pos))
    cadd_table = cadd_table[cadd_table["Alt"] == alt][cadd_anno]
    if not len(cadd_table):
        return [""] * len(cadd_anno)

    # TODO pick the most impactful (min or max) for each variant
    cadd = cadd_table.iloc[0].array.astype(str).tolist()

    # Replace "nan" with ""
    svi = np.where(cadd_table.columns == "SIFTval")[0][0]
    if cadd[svi] == "nan": cadd[svi] = ""
    pvi = np.where(cadd_table.columns == "PolyPhenVal")[0][0]
    if cadd[pvi] == "nan": cadd[pvi] = ""

    # Replace SIFT with categoriacal
    sci = np.where(cadd_table.columns == "SIFTcat")[0][0]
    sc = to_cat(cadd[sci], "SIFTcat").values.reshape(-1).astype(str).tolist()
    cadd[sci] = sc[0]
    for e in range(len(sc[1:])): cadd.insert(sci+e,sc[e])

    # Replace PolyPhen with categorical    
    pci = np.where(cadd_table.columns == "PolyPhenCat")[0][0] + len(sc[1:])
    pc = to_cat(cadd[pci], "PolyPhenCat").values.reshape(-1).astype(str).tolist()
    cadd[pci]=pc[0]
    for e in range(len(pc[1:])): cadd.insert(pci+e,pc[e])
    return cadd

if __name__ == "__main__":
    argParser = argparse.ArgumentParser()
    argParser.add_argument("--pop", default="AFR", type=str)
    argParser.add_argument("--data_dir", default='/oak/stanford/groups/smontgom/erobb/data/watershed', type=str)
    argParser.add_argument("--cadd", default="/oak/stanford/groups/smontgom/erobb/data/watershed/whole_genome_SNVs_inclAnno.tsv.gz", type=str)
    args = argParser.parse_args()
    
    cadd_anno = ["GC", "CpG", "SIFTcat", "SIFTval","PolyPhenCat",
             "PolyPhenVal", "bStatistic", "priPhCons","mamPhCons","verPhCons",
             "priPhyloP","mamPhyloP","verPhyloP","GerpN","GerpS","PHRED"]

### eOutlier

In [ ]:
if __name__ == "__main__":
    #eout_file = f"{data_dir}/eOutlier_scores_{pop}_t3.txt"
    eout_file = f"/oak/stanford/groups/smontgom/erobb/data/watershed/{pop}_exprResiduals.tsv"
    eout_keep_file = f"{data_dir}/ids_outlier_filtered_{pop}_t3f75.txt"

    # Expression outlier scores from residuals file
    eout_file = f"/oak/stanford/groups/smontgom/erobb/data/watershed/{pop}_exprResiduals.tsv"
    eout_keep_file = f"{data_dir}/ids_outlier_filtered_{pop}_t3f75.txt"
    eout_df = pd.read_table(eout_file, sep="\t", index_col=0).T
    # drop transcript information
    gene_names = [c.split(".")[0] for c in eout_df.columns]
    assert(len(np.unique(gene_names)) == len(eout_df.columns))
    eout_df.columns = gene_names
    inds_keep = pd.read_table(eout_keep_file, sep=" ", index_col=0, header=None).T
    eout_df = eout_df.loc[eout_df.index.intersection(inds_keep.columns)]

### gencode

In [37]:
import numpy as np
import os
import pandas as pd
import numpy as np
import argparse
from tqdm import tqdm
import os

def binsearch(df, pos, c1, c2):
    """
    Assumes dataframe is sorted by c1,c2.
    """
    n = len(df)
    if n == 0: return None
    if pos > df[c2].iloc[n//2]: # pos is to the right of "end"
        return binsearch(df.iloc[n//2+1:], pos, c1, c2)
    elif pos < df[c1].iloc[n//2]: # pos is to the left of "start"
        return binsearch(df.iloc[:n//2], pos, c1, c2)
    elif df[c1].iloc[n//2] <= pos and df[c2].iloc[n//2] >= pos:
        return df.iloc[n//2]
    else:
        return None

def add_gencode(df, gencode):
    gc = pd.read_table(gencode, header=None)
    gc = gc.sort_values([0,3,4])
    gc = gc.set_index([0],drop=False)
    
    def _query(row):
        ch, pos = row.index.unique()[0]
        gc_ch_pos = binsearch(gc.loc[f"chr{ch}"], pos, 3, 4)
        if gc_ch_pos is None: return [np.nan, np.nan]
        TSS = pos - gc_ch_pos[3] #gc.loc[w,3].values[0]
        TES = gc_ch_pos[4] - pos #gc.loc[w,4].values[0] - pos
        return [TSS, TES]

    ann = []
    for idx in tqdm(df.index):
        ann.append(_query(df.loc[idx]))
    ann = np.array(ann)

    df.insert(4, "distTSS", ann[:,0])
    df.insert(5, "distTES", ann[:,1])
    return df


if __name__ == '__main__':

    argParser = argparse.ArgumentParser()
    # argParser.add_argument("--pop", default="ESN", type=str)
    # argParser.add_argument("--data_dir", default='/oak/stanford/groups/smontgom/erobb/data/watershed', type=str)
    # argParser.add_argument("--gencode", default="/oak/stanford/groups/smontgom/erobb/data/watershed/gencode.v43.chr_patch_hapl_scaff.annotation.exons.protein_lincRNA.gtf", type=str)
    #args = argParser.parse_args()
    # pop = args.pop
    
    pop = "ESN"
    gencode = "/oak/stanford/groups/smontgom/erobb/data/watershed/gencode.v43.chr_patch_hapl_scaff.annotation.exons.protein_lincRNA.gtf"
    data_dir = '/oak/stanford/groups/smontgom/erobb/data/watershed'
    tsv_in = f'AF.all.{pop}.hg38a.ID.ba.VEP.rare.ws.tsv'
    tsv_out =  f'AF.all.{pop}.hg38a.ID.ba.VEP.gencode.rare.ws.tsv'
    tsv_file = f'{data_dir}/{tsv_in}'
    tsv_file_out = f'{data_dir}/{tsv_out}'


    var_df = pd.read_table(tsv_file)
    var_df = var_df.sort_values(['Chromosome', 'Position'])
    var_df = var_df.set_index(["Chromosome", "Position"], drop=False)

    
    gdf = add_gencode(var_df, gencode)



  0%|                                                                                  | 0/650090 [00:00<?, ?it/s]


> /tmp/ipykernel_51291/1069628940.py(42)add_gencode()
     40         break
     41     import pdb; pdb.set_trace()
---> 42     ann = pd.concatenate(ann)
     43 
     44     df.insert(4, "distTSS", ann[0].values)

ipdb> ann
[[nan, nan]]
ipdb> df.iloc[:1]
                    SubjectID         GeneName  Chromosome  Position    AF  \
Chromosome Position                                                          
1          597782     HG03265  ENSG00000230021           1    597782  0.01   

                     3_prime_UTR_variant  5_prime_UTR_variant  \
Chromosome Position                                             
1          597782                      0                    0   

                     TF_binding_site_variant  downstream_gene_variant  \
Chromosome Position                                                     
1          597782                          0                        1   

                     intergenic_variant  ...  priPhCons  mamPhCons  verPhCons  \
Chromosome P

<stdin>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


ipdb> df
                    SubjectID         GeneName  Chromosome  Position    AF  \
Chromosome Position                                                          
1          597782     HG03265  ENSG00000230021           1    597782  0.01   
           597782     HG03267  ENSG00000230021           1    597782  0.01   
           597782     HG02976  ENSG00000230021           1    597782  0.01   
           630096     HG03301  ENSG00000237973           1    630096  0.01   
           630096     HG03301  ENSG00000198744           1    630096  0.01   
...                       ...              ...         ...       ...   ...   
5          60891442   HG02973  ENSG00000049167           5  60891442  0.01   
           60893612   HG03372  ENSG00000049167           5  60893612  0.01   
           60893612   HG03297  ENSG00000049167           5  60893612  0.01   
           60893612   HG03117  ENSG00000049167           5  60893612  0.01   
           60893612   HG03366  ENSG00000049167         

ipdb> dfp
                    SubjectID         GeneName  Chromosome  Position  TSS  \
Chromosome Position                                                         
1          597782     HG03265  ENSG00000230021           1    597782  NaN   

                     distTES    AF  3_prime_UTR_variant  5_prime_UTR_variant  \
Chromosome Position                                                            
1          597782        NaN  0.01                    0                    0   

                     TF_binding_site_variant  ...  priPhCons  mamPhCons  \
Chromosome Position                           ...                         
1          597782                          0  ...      0.046      0.046   

                     verPhCons  priPhyloP  mamPhyloP  verPhyloP  GerpN  GerpS  \
Chromosome Position                                                             
1          597782        0.297        0.0        0.0        0.0    0.0    0.0   

                     PHRED  eOutlier  
Chromoso

AttributeError: module 'pandas' has no attribute 'concatenate'

In [32]:
print(597782)
#gc = pd.read_table(gencode, header=None)
gc = gc.sort_values([3,4])
gc.loc["chr1"].iloc[1:15]

597782


,0,1,2,3,4,5,6,7,8
0,,,,,,,,,
chr1,chr1,HAVANA,exon,65520,65573,.,+,.,"gene_id ""ENSG00000186092.7""; transcript_id ""EN..."
chr1,chr1,HAVANA,exon,69037,71585,.,+,.,"gene_id ""ENSG00000186092.7""; transcript_id ""EN..."
chr1,chr1,HAVANA,exon,450740,451678,.,-,.,"gene_id ""ENSG00000284733.2""; transcript_id ""EN..."
chr1,chr1,HAVANA,exon,685716,686654,.,-,.,"gene_id ""ENSG00000284662.2""; transcript_id ""EN..."
chr1,chr1,HAVANA,exon,923923,924948,.,+,.,"gene_id ""ENSG00000187634.13""; transcript_id ""E..."
chr1,chr1,HAVANA,exon,923923,924948,.,+,.,"gene_id ""ENSG00000187634.13""; transcript_id ""E..."
chr1,chr1,HAVANA,exon,925150,925189,.,+,.,"gene_id ""ENSG00000187634.13""; transcript_id ""E..."
chr1,chr1,HAVANA,exon,925731,925800,.,+,.,"gene_id ""ENSG00000187634.13""; transcript_id ""E..."
chr1,chr1,HAVANA,exon,925922,926013,.,+,.,"gene_id ""ENSG00000187634.13""; transcript_id ""E..."


In [80]:
pos=597782
ch=1

In [81]:
# gc = pd.read_table(gencode, header=None)
# gc = gc.set_index([0,3,4],drop=False)
w = (gc[0] == f"chr{ch}") & (gc[3] <= pos) & (gc[4] >= pos)

TSS = pos - gc.loc[w,3].values[0]
TES = gc.loc[w,4].values[0] - pos

IndexError: index 0 is out of bounds for axis 0 with size 0

In [86]:
gc.loc[gc[0] == "chr1"]

0       1     2          3          4  5  6  7  \
0    3         4                                                              
chr1 65419     65433      chr1  HAVANA  exon      65419      65433  .  +  .   
     65520     65573      chr1  HAVANA  exon      65520      65573  .  +  .   
     69037     71585      chr1  HAVANA  exon      69037      71585  .  +  .   
     450740    451678     chr1  HAVANA  exon     450740     451678  .  -  .   
     685716    686654     chr1  HAVANA  exon     685716     686654  .  -  .   
...                        ...     ...   ...        ...        ... .. .. ..   
     248913816 248913879  chr1  HAVANA  exon  248913816  248913879  .  +  .   
     248916602 248919146  chr1  HAVANA  exon  248916602  248919146  .  +  .   
     248906372 248906466  chr1  HAVANA  exon  248906372  248906466  .  +  .   
     248913816 248913879  chr1  HAVANA  exon  248913816  248913879  .  +  .   
     248917279 248917401  chr1  HAVANA  exon  248917279  248917401  .  +  .   

                                                                          8  
0    3         4                                                             
chr1 65419     65433      gene_id "ENSG00000186092.7"; transcript_id "EN...  
     65520     65573      gene_id "ENSG00000186092.7"; transcript_id "EN...  
     69037     71585      gene_id "ENSG00000186092.7"; transcript_id "EN...  
     450740    451678     gene_id "ENSG00000284733.2"; transcript_id "EN...  
     685716    686654     gene_id "ENSG00000284662.2"; transcript_id "EN...  
...                                                                     ...  
     248913816 248913879  gene_id "ENSG00000185220.12"; transcript_id "E...  
     248916602 248919146  gene_id "ENSG00000185220.12"; transcript_id "E...  
     248906372 248906466  gene_id "ENSG00000185220.12"; transcript_id "E...  
     248913816 248913879  gene_id "ENSG00000185220.12"; transcript_id "E...  
     248917279 248917401  gene_id "ENSG00000185220.12"; transcript_id "E...  

[132133 rows x 9 columns]

### phyloP

In [ ]:
# 9gb file
!wget http://hgdownload.soe.ucsc.edu/goldenPath/hg38/phyloP100way/hg38.phyloP100way.bw
!wget http://hgdownload.soe.ucsc.edu/admin/exe/linux.x86_64/bigWigToBedGraph

In [ ]:
!chmod +x bigWigToBedGraph
!./bigWigToBedGraph hg38.phyloP100way.bw hg38.phyloP100way.bed

In [35]:
import numpy as np
import os
import pandas as pd
import numpy as np
import argparse
from tqdm import tqdm
import os
import pyBigWig as pbw

def filelen(file):
    fname = file.split("/")[-1]
    os.system(f"wc -l {file} > /tmp/wc_{fname}")
    n = int(open(f"/tmp/wc_{fname}", "r").readlines()[0].split()[0])
    return n

def add_phyloP(df, phylop):
    df = df.sort_values(['Chromosome', 'Position'])
    df = df.set_index(["Chromosome", "Position"], drop=False)
    bw = pbw.open(phylop)
    
    def _query(row): 
        ch, pos = row.name
        return bw.values(f"chr{ch}", pos, pos+1)[0]

    pp = df.apply(_query, 
             axis=1, 
             result_type='expand')
    df.insert(4, "pp", pp)
    return df
    
if __name__ == '__main__':
    # argParser = argparse.ArgumentParser()
    # argParser.add_argument("--pop", default="ESN", type=str)
    # argParser.add_argument("--data_dir", default='/oak/stanford/groups/smontgom/erobb/data/watershed', type=str)
    # argParser.add_argument("--gencode", default="/oak/stanford/groups/smontgom/erobb/data/watershed/gencode.v43.chr_patch_hapl_scaff.annotation.exons.protein_lincRNA.gtf", type=str)
    # args = argParser.parse_args()
    # pop = args.pop

    pop = "ESN"
    phylop = '/oak/stanford/groups/smontgom/erobb/data/hg38.phyloP100way.bw'
    data_dir = '/oak/stanford/groups/smontgom/erobb/data/watershed'
    tsv_in = f'AF.all.{pop}.hg38a.ID.ba.VEP.rare.ws.tsv'
    tsv_out =  f'AF.all.{pop}.hg38a.ID.ba.VEP.gencode.phyloP.rare.ws.tsv'
    tsv_file = f'{data_dir}/{tsv_in}'
    tsv_file_out = f'{data_dir}/{tsv_out}'

    var_df = pd.read_table(tsv_file)
    
    pp_df = add_phyloP(var_df, phylop)


In [36]:
pp_df

SubjectID         GeneName  Chromosome  Position     pp  \
Chromosome Position                                                           
1          597782     HG03265  ENSG00000230021           1    597782  0.953   
           597782     HG03267  ENSG00000230021           1    597782  0.953   
           597782     HG02976  ENSG00000230021           1    597782  0.953   
           630096     HG03301  ENSG00000237973           1    630096 -1.431   
           630096     HG03301  ENSG00000198744           1    630096 -1.431   
...                       ...              ...         ...       ...    ...   
5          60891442   HG02973  ENSG00000049167           5  60891442 -0.127   
           60893612   HG03372  ENSG00000049167           5  60893612  0.520   
           60893612   HG03297  ENSG00000049167           5  60893612  0.520   
           60893612   HG03117  ENSG00000049167           5  60893612  0.520   
           60893612   HG03366  ENSG00000049167           5  60893612  0.520   

                       AF  3_prime_UTR_variant  5_prime_UTR_variant  \
Chromosome Position                                                   
1          597782    0.01                    0                    0   
           597782    0.01                    0                    0   
           597782    0.01                    0                    0   
           630096    0.01                    0                    0   
           630096    0.01                    0                    0   
...                   ...                  ...                  ...   
5          60891442  0.01                    0                    0   
           60893612  0.01                    0                    0   
           60893612  0.01                    0                    0   
           60893612  0.01                    0                    0   
           60893612  0.01                    0                    0   

                     TF_binding_site_variant  downstream_gene_variant  ...  \
Chromosome Position                                                    ...   
1          597782                          0                        1  ...   
           597782                          0                        1  ...   
           597782                          0                        1  ...   
           630096                          0                        0  ...   
           630096                          0                        0  ...   
...                                      ...                      ...  ...   
5          60891442                        0                        1  ...   
           60893612                        0                        1  ...   
           60893612                        0                        1  ...   
           60893612                        0                        1  ...   
           60893612                        0                        1  ...   

                     priPhCons  mamPhCons  verPhCons  priPhyloP  mamPhyloP  \
Chromosome Position                                                          
1          597782        0.046      0.046      0.297      0.000      0.000   
           597782        0.046      0.046      0.297      0.000      0.000   
           597782        0.046      0.046      0.297      0.000      0.000   
           630096        0.007      0.007      0.007      0.000      0.000   
           630096        0.007      0.007      0.007      0.000      0.000   
...                        ...        ...        ...        ...        ...   
5          60891442      0.682      0.001      0.001      0.321      0.504   
           60893612      0.162      0.471      0.477      0.404      0.698   
           60893612      0.162      0.471      0.477      0.404      0.698   
           60893612      0.162      0.471      0.477      0.404      0.698   
           60893612      0.162      0.471      0.477      0.404      0.698   

                     verPhyloP  GerpN 